In [ ]:
#importing all required libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
df = pd.read_excel('Input.xlsx')    #reading input file

In [ ]:
title_name = []
text_content = []
for i in df['URL']:
  html_text = requests.get(f'{i}').text
  soup = BeautifulSoup(html_text,'lxml')
  title = soup.find('h1',attrs={'class','entry-title','tdb-title-text'})            #accessing the title
  content=soup.findAll('div', attrs={'class','td-post-content','tbd-block-inner'})  # accessing the content
  if title == None:
    title_name.append('Data Not Available')
    text_content.append('Data Not Available')
  else:
    title_name.append(title.text)
    content=content[0].text.replace('\n'," ")
    text_content.append(content)


In [ ]:
df = pd.DataFrame({'title_name':title_name,'content':text_content})    # making dataframe

In [ ]:
df.head()

,title_name,content
0,Rise of telemedicine and its Impact on Livelih...,"Telemedicine, the use of technology to diagno..."
1,Rise of e-health and its impact on humans by t...,"The rise of e-health, or the use of electroni..."
2,Rise of e-health and its impact on humans by t...,2020 was the year the world was ravaged by t...
3,Rise of telemedicine and its Impact on Livelih...,"“More gains on quality, affordability and acc..."
4,Rise of telemedicine and its Impact on Livelih...,"“More gains on quality, affordability and acc..."


In [ ]:
df.shape

(114, 2)

In [ ]:
# converting the text into lowercase
def con_to_lower(text):
  return text.lower()

In [ ]:
df['content'] =  df['content'].apply(con_to_lower)

In [ ]:
# removing punctuation
import string
def remove_punc(text):
  return text.translate(str.maketrans('', '',string.punctuation))


In [ ]:
df['content'] =  df['content'].apply(remove_punc)

In [ ]:
df['content'][9]

' “society can exist without marketing but marketing cannot exist without society” marketing is the management process of anticipating identifying and satisfying customers’ requirements the various conventional marketing tools are advertising branding direct marketing sales promotion publicity and public relations critics acknowledge that marketing has legitimate uses in as much as it connects goods and services to consumers who desire them however some aspects of marketing especially promotion and advertisements are subject to criticism they argue that product promotion is an attempt by the goods and service providers to influence demand advertising has become such an inextricable part of our lives that one cannot imagine life without it although we hardly ever notice advertisements leave an indelible mark on our minds especially on vulnerable groups like children and adolescents   the commensuration of the two words raises eyebrows in marketing and society as it is highly debatable o

In [ ]:
import re            #using regular expression
def clean_text(text):
  text = re.sub('[”“–’‘]','',text)
  text = re.sub('\xa0','',text)
  return text


In [ ]:
df['content'] = df['content'].apply(clean_text)

In [ ]:
#tokenization
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
def tokenization(text):
  return word_tokenize(text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df['content'] = df['content'].apply(tokenization)


In [ ]:
# removal of stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
def remove_stopwords(text):
  return [word for word in text if not word in stop_words]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df['content']=df['content'].apply(remove_stopwords)

In [ ]:
#calculating the positive score
positive_score = []
for i in df['content']:
  with open("positive-words.txt","r") as pos:
    posi_words = pos.read().split("\n")
  pos_count = " ".join ([w for w in i if w in posi_words])
  pos_count=pos_count.split(" ")
  Positive_score=len(pos_count)
  positive_score.append(Positive_score)

In [ ]:
# calculating the negative score
negative_score = []
for i in df['content']:
  with open("negative-words.txt","r",encoding = "ISO-8859-1") as neg:
    negwords = neg.read().split("\n")
  neg_count = " ".join ([w for w in i if w in negwords])
  neg_count=neg_count.split(" ")
  Negative_score=len(neg_count)
  negative_score.append(Negative_score)


In [ ]:
# for calculating polarity and Subjectivity
Polarity = []
Subjectivity = []
from textblob import TextBlob
for i in text_content:
    sentiment = TextBlob(i).sentiment

    Polarity.append(sentiment.polarity)
    Subjectivity.append(sentiment.subjectivity)

In [ ]:
#calculating the average sentence length
average_length = []
for i in text_content:
  avg_len = len(i.replace(' ',''))/len(re.split(r'[?!.]', i))
  average_length.append(avg_len)

In [ ]:
#calculating the percentage of complex words
PER_COMPLEX_WORDS = []
for word in text_content:
    count = 0
    vowels = "AEIOUYaeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word.endswith("es"or "ed"):
                count -= 1
    if count == 0:
        count += 1
    PER_COMPLEX_WORDS.append((count*100)/len(word))


In [ ]:
pip install textstat

In [ ]:
# calculating fog index
import textstat
fog_index = []
for i in text_content:
  FOG_INDEX=(textstat.gunning_fog(i))
  fog_index.append(FOG_INDEX)



In [ ]:
# average number of words per sentence
avg_num_of_words = []
for i in text_content:
  n = len(i.replace(' ',''))/len(i.split(' '))
  avg_num_of_words.append(n)

In [ ]:
#complex word count
complex_word_count = []
for word in text_content:
    count = 0
    vowels = "AEIOUYaeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word.endswith("es"or "ed"):
                count -= 1
    if count == 0:
        count += 1
    complex_word_count.append(count)


In [ ]:
#word count
word_count = []
for i in text_content:
  word_count.append(len(i.split(' ')))

In [ ]:
#syllable per word
syll = []
for i in text_content:
  word=i.replace(' ','')
  syllable_count=0
  for w in word:
    if(w=='a' or w=='e' or w=='i' or w=='o' or w=='y' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U' or w=='Y'):
      syllable_count=syllable_count+1
  syll.append(syllable_count/len(i.split()))



In [ ]:
#personal pronouns
personal_pronouns = []
for text in text_content:
    count = 0
    sentences = nltk.sent_tokenize(text)
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        tagged = nltk.pos_tag(words)
        for (word, tag) in tagged:
            if tag == 'PRP': # If the word is a proper noun
                count = count + 1

    personal_pronouns.append(count)

In [ ]:
#average word length
avg_word_len = []
for i in text_content:
  n = len(i.replace(' ',''))/len(i.split(' '))
  avg_word_len.append(n)

In [ ]:
output = pd.read_excel('Output Data Structure.xlsx')

In [ ]:
output.head(0)

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH


In [ ]:
output['POSITIVE SCORE'] = positive_score
output['NEGATIVE SCORE'] = negative_score
output['POLARITY SCORE'] = Polarity
output['SUBJECTIVITY SCORE'] = Subjectivity
output['AVG SENTENCE LENGTH'] = average_length
output['PERCENTAGE OF COMPLEX WORDS'] = PER_COMPLEX_WORDS
output['FOG INDEX'] = fog_index
output['AVG NUMBER OF WORDS PER SENTENCE']  = avg_num_of_words
output['COMPLEX WORD COUNT'] = complex_word_count
output['WORD COUNT'] = word_count
output['SYLLABLE PER WORD'] = syll
output['PERSONAL PRONOUNS'] = personal_pronouns
output['AVG WORD LENGTH'] = avg_word_len


In [ ]:
# some websites in the input.xlsx file  having error
p = 0
for i in text_content:
  if i == 'Data Not Available':
    print(p)           # finding the index of links having error
    p+=1
  else:
    p+=1



24
37


In [ ]:
output.loc[[24,37],'POSITIVE SCORE':] = 'Data Not Available'

In [ ]:
output.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,86,24,0.136034,0.43728,109.321839,30.747204,10.34,5.712312,3436,1665,2.298256,27,5.712312
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,41,13,0.111801,0.615704,135.96,30.475953,14.64,5.526829,1223,615,2.253268,16,5.526829
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,25,27,0.086835,0.459666,79.337838,28.875467,9.78,5.416052,2008,1084,2.076781,13,5.416052
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,44,27,0.139706,0.385624,110.269841,29.048434,12.67,5.647967,2375,1230,2.175325,20,5.647967
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,44,27,0.139706,0.385624,110.269841,29.048434,12.67,5.647967,2375,1230,2.175325,20,5.647967


In [ ]:
output.to_csv('final_output.csv')    #downloading the final output